# Logistic Regression

In [ ]:
library("palmerpenguins")
library("ggplot2")
library("reshape2")
library("tibble")
library("dplyr")
library("purrr")
library("tidyr")

penguins <- penguins[complete.cases(penguins), ]
d <- penguins %>% dplyr::select(c("flipper_length_mm", "bill_depth_mm", "species"))
d %>% head()

In [ ]:
d <- d %>% dplyr::filter(species %in% c("Adelie", "Chinstrap"))
d %>% head()

In [ ]:
d <- d %>% mutate(species = droplevels(species))

In [ ]:
table(d$species)

In [ ]:
d <- d %>% mutate(species = relevel(species, ref = "Adelie"))
d %>% head()

In [ ]:
ggplot(data = d, mapping = aes(x = flipper_length_mm, y = bill_depth_mm, color = species)) +
  geom_point()

In [ ]:
?glm

In [ ]:
log_mod <- glm(formula = species ~ ., data = d, family = "binomial")
summary(log_mod)

recall:

$p = logistic(X\beta) \leftrightarrow logit(p) = log(p/(1-p)) = X\beta$

In [ ]:
head(predict(log_mod)) ## predicts the log-odds = logit(p)

In [ ]:
X_design <- model.matrix(log_mod)
head(X_design)

In [ ]:
beta_hat <- log_mod$coefficients
beta_hat <- array(beta_hat, c(3, 1))
beta_hat

In [ ]:
head(X_design %*% beta_hat)

In [ ]:
p_hats <- predict(log_mod, type = "response")
head(p_hats)

In [ ]:
head(predict(log_mod))
head(log(p_hats / (1 - p_hats)))

In [ ]:
train_pred <- as.numeric(p_hats > .5)
sample(train_pred) %>% head()

In [ ]:
train_pred <- factor(train_pred, labels = c("Adelie", "Chinstrap"))
head(train_pred)

In [ ]:
library("caret")
confusionMatrix(data = train_pred, reference = d$species)

# Can we do this ourselves!?

In [ ]:
y <- as.numeric(d$species) - 1
y %>%
  sample() %>%
  head()

In [ ]:
X <- model.matrix(log_mod)
X %>% head()

In [ ]:
cross_entropy <- function(p, q) {
  return(-(p * log(q) + (1 - p) * log(1 - q)))
}

In [ ]:
logistic <- function(z) 1 / (1 + exp(-z))

In [ ]:
NLL <- function(beta) {
  beta <- array(beta, c(3, 1))
  p <- as.vector(logistic(X %*% beta))
  sum(map2_vec(y, p, ~ cross_entropy(..1, ..2)))
}

In [ ]:
initial_params <- c(0, 0, 0)
result <- optim(par = initial_params, fn = NLL)

In [ ]:
result

In [ ]:
result$par

In [ ]:
as.vector(beta_hat)

# Plotting

In [ ]:
plot_fit_logistic <- function(v1, v2, df = penguins, N = floor(sqrt(10000)), scaleit = TRUE, fmla = "species~.") {
  train_df <- df %>% dplyr::select(all_of(c("species", v1, v2)))
  if (scaleit) {
    train_df <- train_df %>% mutate(across(-species, ~ (.x - mean(.x)) / sd(.x)))
  }

  mod <- glm(formula = as.formula(fmla), data = train_df, family = "binomial")

  combinations <- expand_grid(!!!map(train_df %>% dplyr::select(-species), ~ seq(min(.x), max(.x), length.out = N)))
  colnames(combinations) <- c(v1, v2)
  p_hats <- predict(mod, newdata = combinations, type = "response")
  preds <- (p_hats > 1 / 2) * 1
  preds <- factor(preds, labels = c("Adelie", "Chinstrap"))
  combinations$species <- preds

  ggplot(data = combinations, mapping = aes(x = !!sym(v1), y = !!sym(v2), fill = species, shape = species)) +
    geom_tile() +
    geom_point(data = train_df, size = 5) +
    coord_fixed()
}

In [ ]:
plot_fit_logistic(v1 = "flipper_length_mm", v2 = "bill_depth_mm", df = d)

In [ ]:
plot_fit_logistic(
  v1 = "flipper_length_mm", v2 = "bill_depth_mm", df = d,
  fmla = "species~flipper_length_mm+I(flipper_length_mm^2)+bill_depth_mm+I(bill_depth_mm^2)"
)

# Issues:

In [ ]:
library("palmerpenguins")
penguins <- penguins %>% filter(complete.cases(.))
d <- penguins %>% dplyr::select(c("flipper_length_mm", "bill_depth_mm", "species"))
d <- d %>% dplyr::filter(species %in% c("Adelie", "Gentoo"))
d <- d %>% mutate(species = droplevels(species))
d %>% sample_n(5)

In [ ]:
log_mod <- glm(formula = species ~ ., data = d, family = "binomial")
summary(log_mod)

Problem: classes are perfectly separable:

In [ ]:
plot_fit_logistic(v1 = "flipper_length_mm", v2 = "bill_depth_mm", df = d)

# Multivariate logistic regression

In [ ]:
library("nnet")

In [ ]:
?multinom

In [ ]:
library("palmerpenguins")
penguins <- penguins %>% filter(complete.cases(.))
d <- penguins %>% dplyr::select(c("flipper_length_mm", "bill_depth_mm", "species"))
d <- d %>% mutate(species = relevel(species, ref = "Adelie"))
d %>% sample_n(5)

In [ ]:
mod <- multinom(species ~ ., data = d)
summary(mod)

In [ ]:
prob_preds <- predict(mod, type = "probs")

In [ ]:
head(prob_preds)

In [ ]:
class_preds <- predict(mod, type = "class")
head(class_preds)

In [ ]:
plot_fit_mnlogistic <- function(v1, v2, df = penguins, N = floor(sqrt(10000)), scaleit = TRUE, fmla = "species~.") {
  train_df <- df %>% dplyr::select(all_of(c("species", v1, v2)))
  if (scaleit) {
    train_df <- train_df %>% mutate(across(-species, ~ (.x - mean(.x)) / sd(.x)))
  }

  mod <- multinom(as.formula(fmla), data = train_df)

  combinations <- expand_grid(!!!map(train_df %>% dplyr::select(-species), ~ seq(min(.x), max(.x), length.out = N)))
  colnames(combinations) <- c(v1, v2)
  preds <- predict(mod, newdata = combinations, type = "class")
  combinations$species <- preds

  ggplot(data = combinations, mapping = aes(x = !!sym(v1), y = !!sym(v2), fill = species, shape = species)) +
    geom_tile() +
    geom_point(data = train_df, size = 5) +
    coord_fixed()
}

In [ ]:
plot_fit_mnlogistic(v1 = "flipper_length_mm", v2 = "bill_depth_mm", df = d)

In [ ]:
plot_fit_mnlogistic(
  v1 = "flipper_length_mm", v2 = "bill_depth_mm", df = d,
  fmla = "species~flipper_length_mm+bill_depth_mm+I(bill_depth_mm^2)+I(flipper_length_mm^2)"
)